# Indian Takeaway Orders Dataset - Analysis using PySpark

#### Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col, sum, count, first, countDistinct

#### Start Session

In [2]:
spark = SparkSession.builder.appName("MyApp").getOrCreate()
print(spark.version)

3.5.0


## Load CSV Files

#### Restaurant 1

In [3]:
# Load orders file
r1_orders_df = spark.read.csv("restaurant-1-orders.csv", header = True, inferSchema = True)
r1_orders_df.show(5)

+------------+----------------+-------------------+--------+-------------+--------------+
|Order Number|      Order Date|          Item Name|Quantity|Product Price|Total products|
+------------+----------------+-------------------+--------+-------------+--------------+
|       16118|03/08/2019 20:25|      Plain Papadum|       2|          0.8|             6|
|       16118|03/08/2019 20:25|   King Prawn Balti|       1|        12.95|             6|
|       16118|03/08/2019 20:25|        Garlic Naan|       1|         2.95|             6|
|       16118|03/08/2019 20:25|      Mushroom Rice|       1|         3.95|             6|
|       16118|03/08/2019 20:25|Paneer Tikka Masala|       1|         8.95|             6|
+------------+----------------+-------------------+--------+-------------+--------------+
only showing top 5 rows



In [4]:
# Load products price file
r1_prices_df = spark.read.csv("restaurant-1-products-price.csv", header = True, inferSchema = True)
r1_prices_df.show(5)

+-------------+-------------+
|    Item Name|Product Price|
+-------------+-------------+
|   Mint Sauce|          0.5|
|  Lime Pickle|          0.5|
|Mango Chutney|          0.5|
|    Red Sauce|          0.5|
|Onion Chutney|          0.5|
+-------------+-------------+
only showing top 5 rows



#### Restaurant 2

In [5]:
# Load orders file
r2_orders_df = spark.read.csv("restaurant-2-orders.csv", header = True, inferSchema = True)
r2_orders_df.show(5)

+--------+----------------+--------------------+--------+-------------+--------------+
|Order ID|      Order Date|           Item Name|Quantity|Product Price|Total products|
+--------+----------------+--------------------+--------+-------------+--------------+
|   25583|03/08/2019 21:58|Tandoori Mixed Grill|       1|        11.95|            12|
|   25583|03/08/2019 21:58|        Madras Sauce|       1|         3.95|            12|
|   25583|03/08/2019 21:58|       Mushroom Rice|       2|         3.95|            12|
|   25583|03/08/2019 21:58|         Garlic Naan|       1|         2.95|            12|
|   25583|03/08/2019 21:58|             Paratha|       1|         2.95|            12|
+--------+----------------+--------------------+--------+-------------+--------------+
only showing top 5 rows



In [6]:
# Load products price file
r2_prices_df = spark.read.csv("restaurant-2-products-price.csv", header = True, inferSchema = True)
r2_prices_df.show(5)

+-------------+-------------+
|    Item Name|Product Price|
+-------------+-------------+
|Onion Chutney|          0.5|
|   Mint Sauce|          0.5|
|Mango Chutney|          0.5|
|    Red Sauce|          0.5|
|  Lime Pickle|          0.5|
+-------------+-------------+
only showing top 5 rows



## Data Exploration

### Check Schemas

In [7]:
print("Restaurant 1 Orders Schema:")
r1_orders_df.printSchema()
print("Restaurant 2 Orders Schema:")
r2_orders_df.printSchema()
print("Restaurant 1 Prices Schema:")
r1_prices_df.printSchema()
print("Restaurant 2 Prices Schema:")
r2_prices_df.printSchema()

Restaurant 1 Orders Schema:
root
 |-- Order Number: integer (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Item Name: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Product Price: double (nullable = true)
 |-- Total products: integer (nullable = true)

Restaurant 2 Orders Schema:
root
 |-- Order ID: integer (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Item Name: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Product Price: double (nullable = true)
 |-- Total products: integer (nullable = true)

Restaurant 1 Prices Schema:
root
 |-- Item Name: string (nullable = true)
 |-- Product Price: double (nullable = true)

Restaurant 2 Prices Schema:
root
 |-- Item Name: string (nullable = true)
 |-- Product Price: double (nullable = true)



### Number of Total and Unique Orders

In [8]:
print("Restaurant 1 - Total rows:", r1_orders_df.count())
print("Restaurant 1 - Unique orders:", r1_orders_df.select("Order Number").distinct().count())

print("Restaurant 2 - Total rows:", r2_orders_df.count())
print("Restaurant 2 - Unique orders:", r2_orders_df.select("Order ID").distinct().count())

Restaurant 1 - Total rows: 74818
Restaurant 1 - Unique orders: 13397
Restaurant 2 - Total rows: 119183
Restaurant 2 - Unique orders: 19658


### Summary Stats of Quantities and Prices

In [9]:
print("Restaurant 1:")
r1_orders_df.describe(["Quantity", "Product Price"]).show()
print("Restaurant 2:")
r2_orders_df.describe(["Quantity", "Product Price"]).show()

Restaurant 1:
+-------+------------------+-----------------+
|summary|          Quantity|    Product Price|
+-------+------------------+-----------------+
|  count|             74818|            74818|
|   mean|  1.24356438290251|5.286491886982787|
| stddev|0.7982073410496792|  3.3382213559897|
|    min|                 1|              0.5|
|    max|                51|            17.95|
+-------+------------------+-----------------+

Restaurant 2:
+-------+------------------+------------------+
|summary|          Quantity|     Product Price|
+-------+------------------+------------------+
|  count|            119183|            119183|
|   mean|1.2488693857345428| 5.108173145502638|
| stddev|0.7022026750515845|3.2077047387844217|
|    min|                 1|               0.5|
|    max|                20|             17.95|
+-------+------------------+------------------+



### Top Selling Items by Quantity

In [10]:
# Register the DataFrame as a SQL temporary view
r1_orders_df.createOrReplaceTempView("r1_orders")

# SQL query
top_items_r1 = spark.sql("""
    SELECT `Item Name`, SUM(Quantity) AS Total_Quantity
    FROM r1_orders
    GROUP BY `Item Name`
    ORDER BY Total_Quantity DESC
    LIMIT 10
""")

print("Restaurant 1:")
top_items_r1.show()


Restaurant 1:
+--------------------+--------------+
|           Item Name|Total_Quantity|
+--------------------+--------------+
|       Plain Papadum|         10648|
|          Pilau Rice|          6367|
|          Plain Naan|          4983|
|         Garlic Naan|          3318|
|          Plain Rice|          2964|
|        Onion Bhajee|          2749|
|       Mango Chutney|          2504|
|Chicken Tikka Masala|          2473|
|             Chapati|          1935|
|          Mint Sauce|          1840|
+--------------------+--------------+



In [11]:
# Register the DataFrame as a SQL temporary view
r2_orders_df.createOrReplaceTempView("r2_orders")

# SQL query:
top_items_r2 = spark.sql("""
    SELECT `Item Name`, SUM(Quantity) AS Total_Quantity
    FROM r2_orders
    GROUP BY `Item Name`
    ORDER BY Total_Quantity DESC
    LIMIT 10
""")

print("Restaurant 2:")
top_items_r2.show()


Restaurant 2:
+--------------------+--------------+
|           Item Name|Total_Quantity|
+--------------------+--------------+
|       Plain Papadum|         18056|
|          Pilau Rice|         11754|
|                Naan|          8730|
|         Garlic Naan|          4809|
|         Bombay Aloo|          4336|
|       Mango Chutney|          4124|
|Chicken Tikka Masala|          3970|
|         Onion Bhaji|          3965|
|          Plain Rice|          3532|
|       Mushroom Rice|          3424|
+--------------------+--------------+



### Total Revenue per Order

In [12]:
rev_per_order_r1 = spark.sql("""
    SELECT `Order Number`, SUM(Quantity * `Product Price`) AS Revenue
    FROM r1_orders
    GROUP BY `Order Number`
    ORDER BY Revenue DESC
""")

print("Restaurant 1:")
rev_per_order_r1.show(10)

Restaurant 1:
+------------+------------------+
|Order Number|           Revenue|
+------------+------------------+
|        6769|            1242.0|
|        6768|            685.25|
|       15840| 660.4499999999999|
|        9412| 581.9999999999999|
|        9411| 460.7499999999997|
|        9374|432.04999999999995|
|        9413|289.64999999999986|
|        3976|268.49999999999994|
|        9804|231.89999999999995|
|        9414|224.84999999999988|
+------------+------------------+
only showing top 10 rows



In [13]:
rev_per_order_r2 = spark.sql("""
    SELECT `Order ID`, SUM(Quantity * `Product Price`) AS Revenue
    FROM r2_orders
    GROUP BY `Order ID`
    ORDER BY Revenue DESC
""")

print("Restaurant 2:")
rev_per_order_r2.show(10)

Restaurant 2:
+--------+------------------+
|Order ID|           Revenue|
+--------+------------------+
|    7952|             283.3|
|   13246|234.50000000000006|
|   14224|193.45000000000005|
|   19569|183.65000000000003|
|   22328|182.34999999999997|
|   24856|175.95000000000002|
|   21984|171.04999999999998|
|    5170|167.60000000000002|
|   24855|             167.0|
|   17228|166.60000000000002|
+--------+------------------+
only showing top 10 rows



(Change either Order ID or Order Number later)

### First and Last Orders

In [14]:
#r1_orders_df.select(
  #  min("Order Date").alias("First_Order"),
 #   max("Order Date").alias("Last_Order")
#).show()

## Data Cleaning

### Column Names & Data Types

#### Convert Order Date to Timestamp

In [15]:
# Import to_timestamp
r1_orders_df = r1_orders_df.withColumn("Order Date", to_timestamp("Order Date", "dd/MM/yyyy HH:mm"))
r2_orders_df = r2_orders_df.withColumn("Order Date", to_timestamp("Order Date", "dd/MM/yyyy HH:mm"))

# Check
print(f"Restaurant 1: {r1_orders_df.schema['Order Date'].dataType}, Restaurant 2: {r2_orders_df.schema['Order Date'].dataType}")

Restaurant 1: TimestampType(), Restaurant 2: TimestampType()


#### Rename `Order Number` to `Order ID` for Restaurant 1
(Both tables with the same column name)

In [31]:
r1_orders_df = r1_orders_df.withColumnRenamed("Order Number", "Order ID")
r1_orders_df.select("Order ID").show(1)

+--------+
|Order ID|
+--------+
|   16118|
+--------+
only showing top 1 row



### Functions for Printic Specific Order

In [ ]:
# Register the DataFrame as a SQL temporary view
r1_orders_df.createOrReplaceTempView("r1_orders")
r2_orders_df.createOrReplaceTempView("r2_orders")

In [37]:
def check_order_r1(order_id):
    order = spark.sql(f"""
        SELECT * FROM r1_orders
        WHERE `Order ID` = {order_id}
    """)
    order.show()

def check_order_r2(order_id):
    order = spark.sql(f"""
        SELECT * FROM r2_orders
        WHERE `Order ID` = {order_id}
    """)
    order.show()

### Check for Missing Values

In [17]:
dfs = {
    "Restaurant 1 Orders": r1_orders_df,
    "Restaurant 2 Orders": r2_orders_df,
    "Restaurant 1 Products": r1_prices_df,
    "Restaurant 2 Products": r2_prices_df
}

In [18]:
for name, df in dfs.items():
    print(f"\n{name} - Missing values:")
    df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()


Restaurant 1 Orders - Missing values:
+--------+----------+---------+--------+-------------+--------------+
|Order ID|Order Date|Item Name|Quantity|Product Price|Total products|
+--------+----------+---------+--------+-------------+--------------+
|       0|         0|        0|       0|            0|             0|
+--------+----------+---------+--------+-------------+--------------+


Restaurant 2 Orders - Missing values:
+--------+----------+---------+--------+-------------+--------------+
|Order ID|Order Date|Item Name|Quantity|Product Price|Total products|
+--------+----------+---------+--------+-------------+--------------+
|       0|         0|        0|       0|            0|             0|
+--------+----------+---------+--------+-------------+--------------+


Restaurant 1 Products - Missing values:
+---------+-------------+
|Item Name|Product Price|
+---------+-------------+
|        0|            0|
+---------+-------------+


Restaurant 2 Products - Missing values:
+------

### Check for Duplicates

In [20]:
for name, df in dfs.items():
    dup_rows = df.groupBy(df.columns).count().filter(col("count") > 1)
    print(f"\n{name} duplicate rows:")
    dup_rows.show(5)

    # Sum up the duplicate counts
    total_dup_count = dup_rows.select(sum(col("count") - 1).alias("total_duplicates")).collect()[0]["total_duplicates"]
    print(f"Total duplicate rows count: {total_dup_count}")


Restaurant 1 Orders duplicate rows:
+--------+-------------------+------------+--------+-------------+--------------+-----+
|Order ID|         Order Date|   Item Name|Quantity|Product Price|Total products|count|
+--------+-------------------+------------+--------+-------------+--------------+-----+
|    6017|2017-07-12 18:22:00|       Curry|       1|         7.95|             7|    2|
|    3570|2016-10-30 19:50:00|    Vindaloo|       1|         7.95|             9|    2|
|    3780|2016-11-22 19:30:00|       Korma|       1|         8.95|             7|    2|
|    5805|2017-06-24 18:16:00|       Curry|       1|         7.95|             9|    2|
|   13220|2019-02-02 18:19:00|Rogon - Lamb|       1|         8.95|             8|    2|
+--------+-------------------+------------+--------+-------------+--------------+-----+
only showing top 5 rows

Total duplicate rows count: 97

Restaurant 2 Orders duplicate rows:
+--------+-------------------+----------+--------+-------------+--------------

#### Remove Dupliates

In [21]:
deduped_r1_df = r1_orders_df.dropDuplicates()

#Check
print(f"Restaurant 1 Orders: {r1_orders_df.count() - deduped_r1_df.count()} rows removed")

Restaurant 1 Orders: 97 rows removed


In [22]:
deduped_r2_df = r2_orders_df.dropDuplicates()

#Check
print(f"Restaurant 2 Orders: {r2_orders_df.count() - deduped_r2_df.count()} rows removed")

Restaurant 2 Orders: 178 rows removed


#### Total products column - is it correct for all orders?

In [46]:
# Explore individual orders
check_order_r2(8638)

+--------+-------------------+--------------------+--------+-------------+--------------+
|Order ID|         Order Date|           Item Name|Quantity|Product Price|Total products|
+--------+-------------------+--------------------+--------+-------------+--------------+
|    8638|2017-03-14 18:12:00|                Naan|       2|          2.5|             9|
|    8638|2017-03-14 18:12:00|          Pilau Rice|       5|         2.95|             9|
|    8638|2017-03-14 18:12:00|         Garlic Naan|       2|         2.95|             9|
|    8638|2017-03-14 18:12:00|           Saag Rice|       1|         3.95|             9|
|    8638|2017-03-14 18:12:00|       Muttar Paneer|       2|         5.95|             9|
|    8638|2017-03-14 18:12:00|          Tarka Dall|       1|         5.95|             9|
|    8638|2017-03-14 18:12:00|               Curry|       1|         7.95|             9|
|    8638|2017-03-14 18:12:00|Chicken Tikka Karahi|       1|         9.95|             9|
|    8638|

Total products must mean distinct products rather than total quantity

In [44]:
dfs = {
    "Restaurant 1 Orders": r1_orders_df,
    "Restaurant 2 Orders": r2_orders_df,
    "Restaurant 1 Orders Deduped": deduped_r1_df,
    "Restaurant 2 Orders Deduped": deduped_r2_df
}

In [45]:
# Number of products with incorrect total products column
for name, df in dfs.items():
    # Only run if the relevant columns exist (to avoid errors)
    required_cols = {"Order ID", "Item Name", "Quantity", "Total products"}
    if required_cols.issubset(set(df.columns)):
        order_summary = df.groupBy("Order ID").agg(
            countDistinct("Item Name").alias("distinct_products_count"),
            sum("Quantity").alias("total_quantity"),
            first("Total products").alias("total_products_reported")
        )
        mismatched_orders = order_summary.filter(
            col("distinct_products_count") != col("total_products_reported")
        )
        count_mismatches = mismatched_orders.select("Order ID").distinct().count()
        print(f"{name}: Number of orders where distinct products count != total products reported: {count_mismatches}")
    else:
        print(f"{name}: Skipped because required columns not present.")

Restaurant 1 Orders: Number of orders where distinct products count != total products reported: 299
Restaurant 2 Orders: Number of orders where distinct products count != total products reported: 326
Restaurant 1 Orders Deduped: Number of orders where distinct products count != total products reported: 299
Restaurant 2 Orders Deduped: Number of orders where distinct products count != total products reported: 326


Checked if removing duplicates affected this - it didn't



In [47]:
dfs = {
    "Restaurant 1 Orders Deduped": deduped_r1_df,
    "Restaurant 2 Orders Deduped": deduped_r2_df
}

for name, df in dfs.items():
    order_summary = df.groupBy("Order ID").agg(
        countDistinct("Item Name").alias("distinct_products_count"),
        first("Total products").alias("total_products_reported")
    )

    mismatched_orders = order_summary.filter(
        col("distinct_products_count") != col("total_products_reported")
    )

    print(f"\n{name}: Showing mismatched orders")
    mismatched_orders.show(10)


Restaurant 1 Orders Deduped: Showing mismatched orders
+--------+-----------------------+-----------------------+
|Order ID|distinct_products_count|total_products_reported|
+--------+-----------------------+-----------------------+
|    7340|                     11|                     12|
|    2711|                      7|                      8|
|    4489|                      4|                      5|
|    4364|                      8|                      9|
|    5682|                      5|                      6|
|    5110|                      5|                      6|
|    8450|                      2|                      3|
|    2035|                     11|                     12|
|    1808|                      3|                      4|
|     857|                      1|                      4|
+--------+-----------------------+-----------------------+
only showing top 10 rows


Restaurant 2 Orders Deduped: Showing mismatched orders
+--------+-----------------------+--

In [48]:
check_order_r2(2096)

+--------+-------------------+--------------------+--------+-------------+--------------+
|Order ID|         Order Date|           Item Name|Quantity|Product Price|Total products|
+--------+-------------------+--------------------+--------+-------------+--------------+
|    2096|2015-12-16 19:14:00|          Keema Naan|       1|         2.95|             8|
|    2096|2015-12-16 19:14:00|         Kulcha Naan|       1|         2.95|             8|
|    2096|2015-12-16 19:14:00|          Plain Rice|       1|         2.95|             8|
|    2096|2015-12-16 19:14:00|       Peshwari Naan|       1|         2.95|             8|
|    2096|2015-12-16 19:14:00|         Bombay Aloo|       1|         5.95|             8|
|    2096|2015-12-16 19:14:00|           Aloo Gobi|       1|         5.95|             8|
|    2096|2015-12-16 19:14:00|Chicken Tikka Jal...|       1|         8.95|             8|
+--------+-------------------+--------------------+--------+-------------+--------------+

